#Downloading The Required Packages:
- *datasets*: To download and prepare our data.
- *transformers*: To load and train our Whisper model.
- *soundfile*: To pre-process audio files.
- *evaluate* and *jiwer*: To measure the performance of our model.

In [2]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install soundfile
!pip install librosa
!pip install evaluate
!pip install jiwer
!pip install tensorboard
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 31

# Loading The Dataset:

In [3]:
!cd /content/

You need to login to Hugging Face to download some models that require accepting their terms

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

##Loading Common Voice 13:

In [6]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

# Combine both training and validation splits into one since Arabic dataset is small
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="train+validation")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="test")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 4905it [00:00, 49044.08it/s]
Reading metadata...: 10893it [00:00, 55239.84it/s]
Reading metadata...: 17573it [00:00, 60506.22it/s]
Reading metadata...: 28167it [00:00, 61488.23it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10445it [00:00, 88027.46it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7692it [00:00, 76910.17it/s]
Reading metadata...: 19282it [00:00, 99840.57it/s]
Reading metadata...: 36192it [00:00, 105471.63it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14994it [00:00, 92926.90it/s]


In [7]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 38576
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 10445
    })
})


In [8]:
# Removing unwanted features (we only want audio and its transcription)
common_voice = common_voice.remove_columns(["client_id", "path", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 38576
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10445
    })
})


In [11]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([-2.13162821e-14, -1.01252340e-13, -3.55271368e-14, ...,
        1.26173454e-08,  1.54317320e-07,  1.32340631e-07]), 'sampling_rate': 48000}, 'sentence': 'وما أدراك ما يوم الدين'}


# Preparing The dataset:

## Preparing Feature Extractor & Tokenizer:



* Feature Extractor:
  * Transforms audio into 30s clips either by splitting them if longer than 30s or adding silence if less than 30s.
  * Transforms audio to log-mel spectogram which the model expects as input.





In [16]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")



* Tokenizer:
  * Transforms the output of the model (token IDs) to their respective text.





In [17]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

In [9]:
from transformers import WhisperProcessor

# WhisperProcesor has both feature extractor and tokenizer
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Arabic", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

## Getting Dataset Ready:

In [18]:
# We need to change the sample rate from 48KHz to 16KHz since this is what whisper expects
from datasets import Audio

# cast_column makes datasets perform the resampling on the fly when the data is loaded
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [14]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([ 8.73114914e-11, -4.36557457e-11,  1.67347025e-10, ...,
       -2.33121682e-08, -2.25205440e-07,  4.55183908e-08]), 'sampling_rate': 16000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [25]:
audio = common_voice["train"][0]
print(tokenizer(audio["sentence"]))


{'input_ids': [50258, 50276, 50359, 50363, 2407, 15042, 5551, 3215, 23557, 4117, 19446, 7251, 20498, 32748, 9957, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [28]:
def prepare_dataset(data_item):
    audio = data_item["audio"]
    sentence = data_item["sentence"]

    # compute log-Mel input features from input audio array
    data_item["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"])["input_features"][0]

    # encode target text to label ids
    data_item["labels"] = tokenizer(sentence)["input_ids"]
    return data_item

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map:   0%|          | 0/38576 [00:00<?, ? examples/s]